# Objective

- Create agents that are augmentations of LLMs with functions as tools to achieve business objectives.
- Understand how to create function calling agents using `langchain`.


# Setup

## Installation

In [1]:
! pip install -q openai==1.55.3 \
                 langchain==0.3.7 \
                 langchain-openai==0.2.9

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.2 MB/s eta 0:00:00


## Imports

In [2]:
import os
import json
import requests

import pandas as pd

from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage

from langchain_openai import AzureChatOpenAI

In [3]:
with open('config-azure.json') as f:
    configs = f.read()

In [4]:
creds = json.loads(configs)

In [5]:
llm = AzureChatOpenAI(
    azure_endpoint=creds["AZURE_OPENAI_ENDPOINT"],
    api_key=creds["AZURE_OPENAI_KEY"],
    api_version="2024-02-01",
    model="gpt-4o-mini",
    temperature=0
)

# Wrapping Functions as Tools

## Use Case 1: Responding to customer queries using a database

Consider that we are a banking company who wants to allow customers to track their transactions. We want to build an AI agent that acts as an interface between customers who ask natural language questions and customer support personnel who want to answer these questions. Behind the scenes, the agent uses a `pandas` dataframe of customer data to provide answers to these questions.

Let us begin by creating a dataset and attaching a pandas dataframe to it.

In [6]:
data = {
    'transaction_id': ['T2001', 'T2002', 'T2003', 'T2004', 'T2005'],
    'customer_id': ['C004', 'C005', 'C006', 'C007', 'C008'],
    'payment_amount': [150.75, 92.50, 115.30, 70.20, 225.40],
    'payment_date': ['2023-03-15', '2023-03-16', '2023-03-17', '2023-03-18', '2023-03-19'],
    'payment_status': ['Paid', 'Pending', 'Paid', 'Unpaid', 'Paid']
}

df = pd.DataFrame(data)

Le us now define a couple of functions as tools that allow specific queries to be run on the dataframe.

The key thing here is that with function calling, we can define the capabilities of the agent very clearly. Increasing the functionality of the agent amounts to increasing the number of functions defined as tools. Function calling ensures that there is determinism in the agents' behavior, which is the strongest point of this approach.

To define a function as a tool, we use the `@tool` decorator on the function. Two key aspects of the tool definition are: the function docstring and type hints for the arguments. The content of the docstring is used by the LLM attached to the agent in making a decision on which of the functions should be picked given the user input. The type hints of the function allow the LLM to compose function inputs in the correct format.

In [7]:
@tool
def retrieve_payment_status(transaction_id: str) -> str:
    """
    Get payment status of a transaction
    """
    if transaction_id in df.transaction_id.values:
        return json.dumps({'status': df[df.transaction_id == transaction_id].payment_status.item()})
    return json.dumps({'error': 'transaction id not found.'})

In [8]:
@tool
def retrieve_payment_date(transaction_id: str) -> str:
    """
    Get payment date of a transaction
    """
    if transaction_id in df.transaction_id.values:
        return json.dumps({'date': df[df.transaction_id == transaction_id].payment_date.item()})
    return json.dumps({'error': 'transaction id not found.'})

The above tools allow the user (the support executive in this case) to retrieve the payment status and/or the payment date for a particular transaction.  

With the tools defined, we can now declare these as tools available to the LLM and bind the tools to the LLM to create an agent like so:

In [9]:
available_tools = {
    'retrieve_payment_status': retrieve_payment_status,
    'retrieve_payment_date': retrieve_payment_date
}

In [10]:
llm_with_tools = llm.bind_tools(list(available_tools.values()))

Let us now inspect the execution flow of this agent using the following query. We begin by wrapping the query as a `HumanMessage`.

In [11]:
query = "When was transaction T2004 executed?"

messages = [HumanMessage(query)]

This human message is presented to the agent during invocation as in the code cell below. The key difference here is that the agent does not directly attempt to answer the question. Instead, it identifies the correct tool to be called and composes the function call as expected by the function signature.

In [12]:
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

In [13]:
ai_msg.tool_calls

[{'name': 'retrieve_payment_date',
  'args': {'transaction_id': 'T2004'},
  'id': 'call_7t9Mz9m6uHd4KQQodVu2Htgi',
  'type': 'tool_call'}]

As we append the output from the agent at this stage to the messages in the code cell above, notice that the above output has a special role `tool_call`. When an LLM sees this in the message history, it understands which function was called (`retrieve_payment_date` in this case) and what the arguments for the function were.

Another key thing to note is that the LLM does not execute the function call identified in the `tool_call` by itself. However, since the tool selected is among the boquet of tools declared earlier (`available_tools`), we can extract the tool calls as identified in the list above and invoke the tool with the arguments identified by the LLM.

In [14]:
for tool_call in ai_msg.tool_calls:
    selected_tool = available_tools[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In the above code, we are identifying the functions in the tool calls, obtaining their outpts by executing the functions and appending the tool outputs back to the running list of messages.

In [15]:
len(messages)

3

At this stage the message chain has the original `HumanMessage`, the `AIMessage` that has the tool call (not the output) and the `ToolMessage` that hosts the output from the tool call. Let us inspect the tool output.

In [16]:
messages[-1]

ToolMessage(content='{"date": "2023-03-18"}', tool_call_id='call_7t9Mz9m6uHd4KQQodVu2Htgi')

The output above indicates that the tool execution returned the data to be 2021-10-05. With this chain of messages the LLM can now give an answer to the original question. Notice how we are using the LLM here instead of the LLM with tools.

In [17]:
final_response = llm.invoke(messages)

print(final_response.content)

Transaction T2004 was executed on March 18, 2023.


The code snippet below presents an overall execution of this workflow for few different queries.

In [18]:
def agent_response(query, llm, llm_with_tools):

    messages = [HumanMessage(query)]

    ai_msg = llm_with_tools.invoke(messages)

    messages.append(ai_msg)

    for tool_call in ai_msg.tool_calls:
        selected_tool = available_tools[tool_call["name"].lower()]
        tool_output = selected_tool.invoke(tool_call["args"])
        messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

    final_response = llm.invoke(messages)

    return final_response.content

In [19]:
query = "What is the status of transaction T2005?"
print(agent_response(query, llm, llm_with_tools))

The status of transaction T2005 is "Paid."


In [20]:
query = "What is the status of transactions T2005 and T2002?"
print(agent_response(query, llm, llm_with_tools))

The status of the transactions is as follows:
- Transaction T2005: Paid
- Transaction T2002: Pending


While it is good to distinguish the tool calling agent from the main LLM that is answering queries, invoking the agent (i.e., LLM + tools) also gets us to the final answer.

In [21]:
query = "What is the status of transaction T2005?"
print(agent_response(query, llm_with_tools, llm_with_tools))

The status of transaction T2005 is "Paid."


In [22]:
query = "What is the status of transactions T2005 and T2002?"
print(agent_response(query, llm_with_tools, llm_with_tools))

The status of the transactions is as follows:
- Transaction T2005: **Paid**
- Transaction T2002: **Pending**


Like we noted before a strength of the function calling approach is determinism. The capability of the agent is restricted to a pre-defined set of functions. The agent's behavior is restricted to the functions it can execute and hence there is a good amount of control.

In [23]:
query = "Change the status of transaction T2002 to 'Paid'"
print(agent_response(query, llm, llm_with_tools))

I currently don't have the capability to change the status of transactions. However, I can help you retrieve the payment status or payment date of the transaction T2002 if you need that information. Would you like me to do that?


## Use Case 2: API Calls Wrapped as Functions

An interesting application of function calling is its usage in wrapping APIs. Consider the example of a customer-facing business that invested significant effort in building an emotion classifer and a toxicity classifier. These are in-house models developed by the data science team trained on proprietary data. By hosting these models on company servers and exposing API access, these models effectively become tools for an agent.

As an example, let us build an agent that calls models hosted on HuggingFace based on the input received to achieve the objective listed in user instructions.

An extension of this approach is that any machine learning model can be converted into a function call and presented as a tool to an agent to choose from.

In [ ]:
with open('config-hf.json') as f:
    hf_configs = f.read()

hf_creds = json.loads(hf_configs)

In the two tool definitions presented below, we take the text presented by the LLM as an input, call the API hosted on a specific URL and return the answer back. Since these are external APIs, a simple error handling mechanism is implemented.

In [24]:
@tool
def assign_emotion(input: str) -> str:
    """"
    Assign emotion associated with an input text
    """

    API_URL = "https://api-inference.huggingface.co/models/michellejieli/emotion_text_classifier"
    hf_key = hf_creds["HUGGINGFACE_API_KEY"]
    headers = {"Authorization": f"Bearer {hf_key}"}
    payload = {
        "inputs": input
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    try:
        final_emotion = response.json()[0][0]['label']
    except Exception as e:
        print(e)
        final_emotion = 'neutral'

    return final_emotion


In [25]:
@tool
def detect_toxicity(input: str) -> str:
    """"
    Detect if the input text is toxic or neutral.
    """

    API_URL = "https://api-inference.huggingface.co/models/s-nlp/roberta_toxicity_classifier"
    hf_key = hf_creds["HUGGINGFACE_API_KEY"]
    headers = {"Authorization": f"Bearer {hf_key}"}
    payload = {
        "inputs": input
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    try:
        final_label = response.json()[0][0]['label']
    except Exception as e:
        print(e)
        final_label = 'neutral'

    return final_label

With these tools defined, we can bind them to the LLM and create a nenw agent exactly like we did in the previous section.

In [26]:
available_tools = {
    'assign_emotion': assign_emotion,
    'detect_toxicity': detect_toxicity
}

llm_with_tools = llm.bind_tools(list(available_tools.values()))

In [27]:
def answer(query):

    messages = [
        SystemMessage(
            """
            Answer only with the output from tools.
            Note that the tool choice is optional.
            If answer is not available in the tool output say 'I don't know'.
            If the user queries cannot be answered with the tools available say 'I don't know'.
            """
        ),
        HumanMessage(query)
    ]

    ai_msg = llm_with_tools.invoke(messages)
    messages.append(ai_msg)

    for tool_call in ai_msg.tool_calls:
        selected_tool = available_tools[tool_call["name"].lower()]
        tool_output = selected_tool.invoke(tool_call["args"])
        messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

    final_response = llm.invoke(messages)

    return final_response.content

Let us now test our API-calling agent with a few inputs (since these are APIs hosted on free HuggingFace servers, expect errors in execution; multiple runs might be required.)

In [28]:
# Tests
## 1
query = "Is there a hint of sadness in this review? Review: The movie made me feel very sad."
response = answer(query)
print(response)

0
0
I don't know.


In [29]:
## 2
query = "Is the following message toxic? Message: This is a cruel, cruel world"
response = answer(query)
print(response)

0
The message is not toxic.


In [30]:
## 3
query = "Which of 12 * 14 and 14+1000 is the larger number?"
response = answer(query)
print(response)

I don't know.


In [31]:
## 4
query = """
Assign sentiment to the following review.
Before assigning sentiment, please check first if the message is toxic.
If the message is detected to be toxic then do not assign sentiment but say 'Sorry, cannot help you on that'.
Review: This is a cruel, cruel world.
"""

response = answer(query)
print(response)

The sentiment of the review is negative.
